# Project 1
## Step 1: Open the sat_scores.csv file. Investigate the data, and answer the questions below.

1. What does the data describe?

For each US state, plus the District of Columbia, the data covers the percentage of students taking the SAT, and the average(?) verbal and math scores. 

2. Does the data look complete? Are there any obvious issues with the observations?

It appears complete in that there are 51 sets of data points, and none appear to be glaringly anomalous. The wide range of participation rates is a little surprising to me, but since they are somewhat contiguous, rather than clustered with a few outliers, I'll assume it's accurate data. 

Some better definitions would be nice. What students are counted in the rate -- all graduating seniors, perhaps? Are the scores really an average? Does this data represent a single year or multiple years? 

The fact that Nebraska uses an older definition makes me suspect the data itself might be years out of date. That's no guarantee; it could also be an error in the data. If this was data handewd me by a client, I'd ask. 

3. Create a data dictionary for the dataset.

| Field Name | Data Type | Field Length | Constraint                                 | Description                                    |
|------------|-----------|--------------|--------------------------------------------|------------------------------------------------|
| State      | String    | 2            | Must be in list of US postal abbreviations | US state represented by data                   |
| Rate       | Int       | 8 bits       | 0 <= Rate <= 100                           | Percentage of students in state taking the SAT |
| Verbal     | Int       | 8 bits      | 200 <= Verbal <= 800                       | Average(?) Verbal Score                        |
| Math       | Int       | 8 bits      | 200 <= Math <= 800                         | Average(?) Verbal Score                        |

## Step 2: Load the data.
4. Load the data into a list of lists

I'm not happy about the way I'm doing this and hard-coding the column titles. They are stored in what np.genfromtext returns, but I can't come up with a satisfactory way to make it work "nicely", so I'm kind of "brute-forcing" it for now. My instincts are that I'm missing something simple.

In [1]:
# Get all our imports out of the way 
import numpy as np
from numpy import mean, median, std, ptp, var
from scipy.stats import mode, skew
from scipy.optimize import curve_fit

import matplotlib
import matplotlib.pyplot as plt

import math

In [2]:
def buildDictionaries( data ):
    # Go through the CSV data and store as a ist of lists.
    dataLists = { "Rate" : {}, "Verbal" : {}, "Math" : {} }
    for row in data:
        dataLists["Rate"][row[0]] = row[1]
        dataLists["Verbal"][row[0]] = row[2]
        dataLists["Math"][row[0]] = row[3]
    return dataLists

# File where the data lives. 
filename = "sat_scores.csv"

# Read in the file
sat_csv = np.genfromtxt(filename, dtype = None, names = True, delimiter = ",")

dataDict = buildDictionaries( sat_csv )

5. Print the data
This is just a straightforward iteration over a list of lists.
This does bring up one thing I'm not sure i understand about Jupyter notebooks.
I'm "old-school" and like to put all my function definitions at the top of the file,
but these notebooks seem to encourage putting your function definitions where they're
first needed. Is the latter the proper form for Jupyter notebooks?

In [3]:
def listOfListsString( datadict ):
    printable = ""
    
    # Iterate over the first key
    for key1 in datadict:
        # And the second
        for key2 in datadict[key1]:
            # Add to s string showing the data
            printable += str( key1 ) + " : " + str( key2 ) + " : " +\
                str( datadict[key1][key2] ) + "\n"
    # Return the string
    return printable

# Print the data
print listOfListsString( dataDict )

Rate : WA : 53
Rate : DE : 67
Rate : DC : 56
Rate : WI : 6
Rate : WV : 18
Rate : HI : 52
Rate : FL : 54
Rate : WY : 11
Rate : NH : 72
Rate : NJ : 81
Rate : NM : 13
Rate : TX : 53
Rate : LA : 7
Rate : NC : 65
Rate : ND : 4
Rate : NE : 8
Rate : TN : 13
Rate : NY : 77
Rate : PA : 71
Rate : RI : 71
Rate : NV : 33
Rate : VA : 68
Rate : CO : 31
Rate : AK : 51
Rate : AL : 9
Rate : AR : 6
Rate : VT : 69
Rate : IL : 12
Rate : GA : 63
Rate : IN : 60
Rate : IA : 5
Rate : OK : 8
Rate : AZ : 34
Rate : CA : 51
Rate : ID : 17
Rate : CT : 82
Rate : ME : 69
Rate : MD : 65
Rate : MA : 79
Rate : OH : 26
Rate : UT : 5
Rate : MO : 8
Rate : MN : 9
Rate : MI : 11
Rate : KS : 9
Rate : MT : 23
Rate : MS : 4
Rate : SC : 57
Rate : KY : 12
Rate : OR : 55
Rate : SD : 4
Math : WA : 527
Math : DE : 499
Math : DC : 474
Math : WI : 596
Math : WV : 512
Math : HI : 515
Math : FL : 499
Math : WY : 545
Math : NH : 516
Math : NJ : 513
Math : NM : 542
Math : TX : 499
Math : LA : 562
Math : NC : 499
Math : ND : 599
Math : NE

7. Extract a list of the labels from the data, and remove them from the data.
My store method already removes them.

8. Print the types of each column

In [4]:
print str( sat_csv.dtype )


[('State', 'S2'), ('Rate', '<i8'), ('Verbal', '<i8'), ('Math', '<i8')]


9. Do any types need to be reassigned? If so, go ahead and do it.

They look good. The states arew 2-char string, the rate and two scores are all ints. 

10. Create a dictionary for each column mapping the State to its respective value for that column.
Again, I'm hardcoding the names; I don't like doing that. 

In [5]:
def getStates( data ):
    states = []
    for row in data:
        states.append( row[0] )
    return states
    

def buildDictionaries( data ):
    # Create a dictionary out of a list in the order Rate, Verbal, Math
    dataLists = { "Rate" : {}, "Verbal" : {}, "Math" : {} }
    for row in data:
        dataLists["Rate"][row[0]] = row[1]
        dataLists["Verbal"][row[0]] = row[2]
        dataLists["Math"][row[0]] = row[3]
    return dataLists
    
def getAxis( stateKeys, colDict ):
    # Create a dictionary for column's entries, with the state as a key.  
    axis = []
    for state in stateKeys:
        axis.append( colDict[state] )
    return axis

stateList = getStates( sat_csv )

rateAxis = getAxis( stateList, dataDict["Rate"])
verbalAxis = getAxis( stateList, dataDict["Verbal"])
mathAxis = getAxis( stateList, dataDict["Math"])

12. Print the min and max of each column

In [6]:
print "Rate minimum: " + str( min(rateAxis) )
print "Rate maximum: " + str( max(rateAxis) )
print "Verbal minimum: " + str( min(verbalAxis) )
print "Verbal maximum: " + str( max(verbalAxis) )
print "Math minimum: " + str( min(mathAxis) )
print "Math maximum: " + str( max(mathAxis) )

Rate minimum: 4
Rate maximum: 82
Verbal minimum: 482
Verbal maximum: 593
Math minimum: 439
Math maximum: 603


13. Write a function using only list comprehensions, no loops, to compute Standard Deviation. Print the Standard Deviation of each numeric column

In [7]:
def myVariance( data ):
    average = float( sum ( data ) )/float(len (data))
    return sum( [(n - average) ** 2 for n in data])/float( len(data))

def myStandardDeviation( data ):
    return math.sqrt( myVariance(data) )

print "Rate standard deviation: " + str(myStandardDeviation(dataDict["Rate"].values()))
print "Verbal standard deviation: " + str(myStandardDeviation(dataDict["Verbal"].values()))
print "Math standard deviation: " + str(myStandardDeviation(dataDict["Math"].values()))

Rate standard deviation: 27.2792386761
Verbal standard deviation: 33.0319826842
Math standard deviation: 35.9298731731


14. Using MatPlotLib and PyPlot, plot the distribution of the Rate using histograms.

In [8]:
plt.hist(rateAxis)
plt.xlabel("Percentage of Students Taking SAT")
plt.title("Rate Histogram")
plt.show()

![Rate Histogram](rate_histo.png)

15. Plot the Math distribution

In [9]:
plt.hist(mathAxis)
plt.xlabel("Average Math Scores")
plt.title("Math Score Histogram")
plt.show()

![Math Histogram](math_histo.png)

16. Plot the Verbal distribution

In [10]:
plt.hist(verbalAxis)
plt.xlabel("Average Verbal Scores")
plt.title("Verbal Score Histogram")
plt.show()

![Verbal Histogram](verbal_histo.png)

17. What is the typical assumption for data distribution?

That it follows a normal, bell-shapee distributions.

18. Does that distribution hold true for our data?

Not at all! If anything, it's a bit of an inverted bell curve, with a "dip" in the middle for all three.

19. Plot some scatterplots. BONUS: Use a PyPlot figure to present multiple plots at once.

In [11]:
plt.plot( rateAxis, verbalAxis, color="red" )
plt.scatter(rateAxis, verbalAxis, color="red" )
plt.xlabel("Rate")
plt.plot( rateAxis, mathAxis, color="blue" )
plt.scatter(rateAxis,mathAxis, color="blue" )
plt.ylabel("Verbal Score (Red)\nMath Score (Blue)")
plt.show()

![Verbal and Math Scatterplot](verbal_and_math_scatter.png)

In [12]:
plt.scatter(verbalAxis, mathAxis)
plt.xlabel("Verbal Score")
plt.ylabel("Math Score")
plt.show()

![Verbal Scores vs Math Scores](verbal_vs_math_scatter.png)

20. Are there any interesting relationships to note?

Though there's not a strong correlation, it seems the more students take the SAT, the lower the average score. At first, this seemed slightly odd, but I realized that in states where 5% or so of students take the teet, it's likely that the top students are the ones taking the test. In states where a majority, if not a vast majority, are taking the exam, there will be more students who are not top performers. (This is not Lake Wobegon, where all the students are above average1)

21. Create box plots for each variable.

In [18]:
plt.boxplot(rateAxis,0,'rs',0)
plt.title("Rate Boxplot")
plt.show()

![Rate Boxplot](rate_boxplot.png)

In [19]:
plt.boxplot(verbalAxis)
plt.title("Verbal Scores Boxplot")
plt.show()

![Math Scores Boxplot](verbal_boxplot.png)

In [ ]:
plt.boxplot(mathAxis,0,'gD')
plt.title("Math Scores Boxplot")
plt.show()

![Math Scores Boxplot](math_boxplot.png)

BONUS: Using Tableau, create a heat map for each variable using a map of the US.

Done, but how I attach it here? 